In [40]:
import os
import pandas as pd

df = pd.read_csv(os.environ.get('FINETUNE_CSV_PATH'))

In [41]:
df = df.T.reset_index().T
new_columns = ['instruction', 'output']
df.columns = new_columns
df.index = range(0, len(df))
df.head()

,instruction,output
0,give me purchase id from 10th of may,SELECT PURCHASE_ID FROM VAT_PURCHASE WHERE PUR...
1,find the first name and last name of entities ...,"SELECT FIRST_NAME, LAST_NAME FROM JPK_ENTITY W..."
2,show all headers sent after 1st January 2023,SELECT * FROM JPK_HEADER WHERE SEND_TIME > '20...
3,get all supplier numbers who have a purchase d...,SELECT SUPPLIER_NUMBER FROM VAT_PURCHASE WHERE...
4,fetch all purchases from supplier number 12345,SELECT * FROM VAT_PURCHASE WHERE SUPPLIER_NUMB...


In [51]:
with open(f"{os.environ.get('SQL_FILES_PATH')}/ddl.sql", "r") as file:
    ddl = file.read()
df['input'] = ddl

In [63]:
text_col = []

for _, row in df.iterrows():
    prompt = 'Below is an instruction which needs to be turned into SQL query, paired with input that consists of build of this database. Respond with appropriate SQL query for described database'
    instruction = row['instruction']
    input_query = row['input']
    response = row['output']

    text = (
        prompt
        + '### Instruction:\n'
        + instruction
        + '### Input:\n'
        + input_query
        + '### Response:\n'
        + response
    )

    text_col.append(text)

df['text'] = text_col

In [67]:
df[20:].to_csv('train.csv', index=False)
df[:20].to_csv('test.csv', index=False)
